___
___
___
# Question SQL
___
___
___

## INIT
### imports
### connecteur + curseur

In [29]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'azertyuiop',
    database = 'my_imdb'
)

cur = conn.cursor()


___
### 1) Quel est le film le plus long ?

In [30]:
request = """
SELECT CONCAT("Le film le plus long de la BDD est “",
              title,
              "” avec ses ",
              TRUNCATE(duration / 60, 0),
              "h ",
              duration - TRUNCATE(duration / 60, 0) * 60,
              "min")
  FROM movies250 m
 ORDER BY duration DESC
 LIMIT 1;
"""

cur.execute(request)
results = cur.fetchall()

for row in results:
    print(row[0])

Le film le plus long de la BDD est “La flor” avec ses 13h 28min


___
### 2) Quels sont les 5 films les mieux notés ?

In [31]:
x = 5 # nombre de films

request = f"""
SELECT RPAD(CONCAT("“",
              title,
              "”"), (SELECT MAX(title_length)
                       FROM (SELECT LENGTH(title) title_length
                               FROM movies250
                              ORDER BY score DESC
                              LIMIT {x}) sr) + 2, "-"),
       CONCAT(score, "/10")
  FROM movies250 m
 ORDER BY score DESC
 LIMIT {x};
"""

cur.execute(request)
results = cur.fetchall()

print(f"Les {x} films les mieux notés sont (dans l'ordre) :")
rang = 0
for row in results:
    rang += 1
    print(rang, ":", row[0], row[1])

Les 5 films les mieux notés sont (dans l'ordre) :
1 : “The Counterfeit Kid” 9.7/10
2 : “My Dead Friend Zoe”- 9.5/10
3 : “The Home Game”------ 9.5/10
4 : “Desert Road”-------- 9.4/10
5 : “Amaltash”----------- 9.4/10


___
### 3) Dans combien de films a joué Morgan Freeman ? Tom Cruise ?

In [32]:
request = f"""
SELECT IFNULL(ma.actor_name, "TOTAL") Nom, COUNT(1) "Nombre de films"
  FROM movies250 m
  JOIN movie_actor ma ON ma.movie_url  = m.url 
 WHERE ma.actor_name IN ("Morgan Freeman", "Tom Cruise")
 GROUP BY ma.actor_name WITH ROLLUP
 ORDER BY 2 DESC
"""

cur.execute(request)
results = cur.fetchall()

df = pd.DataFrame(results, columns=['Nom', 'Nombre de films'])

display(df)

,Nom,Nombre de films
0,TOTAL,63
1,Tom Cruise,37
2,Morgan Freeman,26


___
### 4) Quels sont les 3 meilleurs films d’horreur ? Dramatique ? Comique ?

In [33]:
genres = '"Comédie", "Drame", "Horreur"'

request = f"""
WITH ranked_movies AS (
                       SELECT m.title Titre, mg.genre_name Genre, m.score Note, 
                              ROW_NUMBER() OVER (PARTITION BY mg.genre_name ORDER BY m.score DESC) AS rank_pos
                         FROM movies250 m 
                         JOIN movie_genre mg ON mg.movie_url = m.url 
                        WHERE mg.genre_name IN ({genres}))
SELECT Titre, Genre, Note
  FROM ranked_movies
 WHERE rank_pos <= 3
"""

cur.execute(request)
results = cur.fetchall()

df = pd.DataFrame(results, columns=['Titre', 'Genre', 'Note'])

display(df)

,Titre,Genre,Note
0,The Counterfeit Kid,Comédie,9.7
1,The Home Game,Comédie,9.5
2,My Dead Friend Zoe,Comédie,9.5
3,My Dead Friend Zoe,Drame,9.5
4,Amaltash,Drame,9.4
5,Amaltash,Drame,9.4
6,Wedding 93,Horreur,9.4
7,Desert Road,Horreur,9.4
8,Tantra,Horreur,8.7


___
### 5) Parmi les 100 films les mieux notés, quel pourcentage sont américains ? Français ?

In [34]:
countries = '"États-Unis", "France"'

request = f"""
SELECT IFNULL(m.country, "TOTAL") Pays, 
       COUNT(1) AS "Nombre de films",
       CONCAT(ROUND(COUNT(1) * 100.0 / (SELECT COUNT(1) FROM movies250), 2), "%") AS Pourcentage
  FROM movies250 m
WHERE m.country IN ({countries})
 GROUP BY m.country WITH ROLLUP
 ORDER BY 3 DESC
"""

cur.execute(request)
results = cur.fetchall()

df = pd.DataFrame(results, columns=['Pays', 'Nombre de films', 'Pourcentage'])

display(df)

,Pays,Nombre de films,Pourcentage
0,TOTAL,6906,65.97%
1,États-Unis,6578,62.84%
2,France,328,3.13%


___
### 6) Quel est la durée moyenne d’un film en fonction du genre ?

In [35]:
genres_excluded = '"Télé-réalité", "Jeu télévisé"'

request = f"""
SELECT mg.genre_name Genre,
       CONCAT(FLOOR(AVG(m.duration) / 60), 'h ', ROUND(MOD(AVG(m.duration), 60), 0), 'min') AS "Durée moyenne"
  FROM movie_genre mg
  JOIN movies250 m ON m.url = mg.movie_url
 WHERE mg.genre_name NOT IN ({genres_excluded})
 GROUP BY mg.genre_name 
 ORDER BY AVG(m.duration) DESC
"""

cur.execute(request)
results = cur.fetchall()

df = pd.DataFrame(results, columns=['Genre', 'Durée moyenne'])

display(df)

,Genre,Durée moyenne
0,Romantique,1h 43min
1,Science-fiction,1h 42min
2,Guerre,1h 41min
3,Action,1h 41min
4,Aventure,1h 40min
5,Biographique,1h 40min
6,Fantastique,1h 40min
7,Drame,1h 39min
8,Mystère,1h 38min
9,Comédie Musicale,1h 38min


___
## Bonus :
### 7) En fonction du genre, afficher la liste des films les plus longs.

In [36]:
nombre_de_film = 1

request = f"""
WITH ranked_movies AS (
                       SELECT mg.genre_name Genre, m.title Titre, m.duration, 
                              ROW_NUMBER() OVER (PARTITION BY mg.genre_name ORDER BY m.duration DESC) AS rank_pos
                         FROM movie_genre mg 
                         JOIN movies250 m ON m.url  = mg.movie_url)
SELECT Genre, Titre, CONCAT(FLOOR(duration / 60), 'h ', ROUND(MOD(duration, 60), 0), 'min') "Durée"
  FROM ranked_movies
 WHERE rank_pos <= {nombre_de_film}
"""

cur.execute(request)
results = cur.fetchall()

phrase1 = f"Les {nombre_de_film} films"
phrase2 = "les plus longs"
if nombre_de_film == 1:
    phrase1 = "Le film"
    phrase2 = "le plus long"

genre = ''
for row in results:
    if genre != row[0]:
        genre = row[0]
        print()
        print(f"{phrase1} “{genre}” {phrase2} :")
    print("   -", row[1], ":", row[2])


Le film “Action” le plus long :
   - FPS: First Person Shooter : 4h 35min

Le film “Actualités” le plus long :
   - Ukraine Is Not a Brothel : 1h 18min

Le film “Animation” le plus long :
   - Batman: The Long Halloween : 2h 48min

Le film “Aventure” le plus long :
   - Flash Gordon : 4h 5min

Le film “Biographique” le plus long :
   - O.J.: Made in America : 7h 47min

Le film “Comédie” le plus long :
   - Salaam-E-Ishq : 3h 36min

Le film “Comédie Musicale” le plus long :
   - La flor : 13h 28min

Le film “Documentaire” le plus long :
   - Shoah : 9h 26min

Le film “Drame” le plus long :
   - La flor : 13h 28min

Le film “Famille” le plus long :
   - Flash Gordon : 4h 5min

Le film “Fantastique” le plus long :
   - La flor : 13h 28min

Le film “Film noir” le plus long :
   - Pour que vivent les hommes : 2h 15min

Le film “Guerre” le plus long :
   - Shoah : 9h 26min

Le film “Historique” le plus long :
   - Shoah : 9h 26min

Le film “Horreur” le plus long :
   - In Search of Darkness

___
### 8) En fonction du genre, quel est le coût de tournage d’une minute de film ?

In [37]:
nombre_de_film = 1

request = f"""
WITH ranked_movies AS (
                       SELECT mg.genre_name Genre, m.title Titre, m.duration, 
                              ROW_NUMBER() OVER (PARTITION BY mg.genre_name ORDER BY m.duration DESC) AS rank_pos
                         FROM movie_genre mg 
                         JOIN movies250 m ON m.url  = mg.movie_url)
SELECT Genre, Titre, CONCAT(FLOOR(duration / 60), 'h ', ROUND(MOD(duration, 60), 0), 'min') "Durée"
  FROM ranked_movies
 WHERE rank_pos <= {nombre_de_film}
"""

cur.execute(request)
results = cur.fetchall()

phrase1 = f"Les {nombre_de_film} films"
phrase2 = "les plus longs"
if nombre_de_film == 1:
    phrase1 = "Le film"
    phrase2 = "le plus long"

genre = ''
for row in results:
    if genre != row[0]:
        genre = row[0]
        print()
        print(f"{phrase1} “{genre}” {phrase2} :")
    print("   -", row[1], ":", row[2])


Le film “Action” le plus long :
   - FPS: First Person Shooter : 4h 35min

Le film “Actualités” le plus long :
   - Ukraine Is Not a Brothel : 1h 18min

Le film “Animation” le plus long :
   - Batman: The Long Halloween : 2h 48min

Le film “Aventure” le plus long :
   - Flash Gordon : 4h 5min

Le film “Biographique” le plus long :
   - O.J.: Made in America : 7h 47min

Le film “Comédie” le plus long :
   - Salaam-E-Ishq : 3h 36min

Le film “Comédie Musicale” le plus long :
   - La flor : 13h 28min

Le film “Documentaire” le plus long :
   - Shoah : 9h 26min

Le film “Drame” le plus long :
   - La flor : 13h 28min

Le film “Famille” le plus long :
   - Flash Gordon : 4h 5min

Le film “Fantastique” le plus long :
   - La flor : 13h 28min

Le film “Film noir” le plus long :
   - Pour que vivent les hommes : 2h 15min

Le film “Guerre” le plus long :
   - Shoah : 9h 26min

Le film “Historique” le plus long :
   - Shoah : 9h 26min

Le film “Horreur” le plus long :
   - In Search of Darkness

___
### 9) Quelles sont les séries les mieux notées ?

In [38]:
request = f"""
SELECT mg.genre_name Genre,
       CONCAT(FORMAT(AVG(m.budget / m.duration), 2), ' $/min') AS "Coût moyen/min",
       CONCAT(FORMAT(STDDEV(m.budget / m.duration), 2), ' $/min') AS "Écart type"
  FROM movie_genre mg
  JOIN movies250 m ON m.url = mg.movie_url
 WHERE m.duration > 0
   AND m.budget IS NOT NULL
 GROUP BY mg.genre_name
 ORDER BY 2 DESC
"""

cur.execute(request)
results = cur.fetchall()

df = pd.DataFrame(results, columns=['Genre', 'Coût moyen/min', 'Écart type'])

display(df)

,Genre,Coût moyen/min,Écart type
0,Aventure,"879,382.22 $/min","4,766,363.79 $/min"
1,Documentaire,"85,624.95 $/min","259,620.25 $/min"
2,Mystère,"759,591.73 $/min","10,563,482.68 $/min"
3,Drame,"752,562.70 $/min","8,404,408.98 $/min"
4,Famille,"576,066.94 $/min","1,976,441.94 $/min"
5,Science-fiction,"567,399.35 $/min","529,429.95 $/min"
6,Fantastique,"523,777.37 $/min","674,754.75 $/min"
7,Comédie,"496,831.49 $/min","4,155,122.76 $/min"
8,Horreur,"355,120.59 $/min","3,722,403.43 $/min"
9,Policier,"336,375.69 $/min","1,522,400.42 $/min"


### fermeture du curseur + connecteur

In [39]:
cur.close()
conn.close()